In [69]:
import numpy as np
import cv2
from collections import Counter
import os

# 1. Сохранение изображения в виде текстового файла.


In [70]:
img_path = 'sar_1_gray.jpg'
# img_path = 'sar_1_gray_mini_copy.jpg'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

np.savetxt('pixel_image.txt', img, fmt='%d')
pixel_storage_size = os.path.getsize('pixel_image.txt')

# 2. Вейвлет-преобразования Хаара


In [71]:
def haar_preobr(image):
    rows, cols = image.shape
    image_float = image.astype(np.float64)
    row_transform = np.zeros_like(image_float)
    
    for i in range(rows):
        for j in range(0, cols - 1, 2):
            row_transform[i, j // 2] = (image_float[i, j] + image_float[i, j + 1]) / 2
            row_transform[i, (j // 2) + cols // 2] = (image_float[i, j] - image_float[i, j + 1]) / 2
    
    result = np.zeros_like(row_transform)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            result[i // 2, j] = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            result[(i // 2) + rows // 2, j] = (row_transform[i, j] - row_transform[i + 1, j]) / 2
    
    LL = result[:rows // 2, :cols // 2]  
    LH = result[:rows // 2, cols // 2:]  
    HL = result[rows // 2:, :cols // 2] 
    HH = result[rows // 2:, cols // 2:]
    return LL, LH, HL, HH

LL, LH, HL, HH = haar_preobr(img)


# 3. Квантование высокочастотных компонент


In [72]:
def quant(coeffs, n_quants):
    min_val = np.min(coeffs)
    max_val = np.max(coeffs)
    step = (max_val - min_val) / n_quants
    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized

LH_q = quant(LH, 4)
HL_q = quant(HL, 4)
HH_q = quant(HH, 4)

# 4. Сохраните получившийся массив значений в текстовый файл

In [73]:
def run_length_encode(data):
    encoded = []
    for value, count in Counter(data.flatten()).items():
        encoded.append((value, count))
    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

with open('haar_image.txt', 'w') as f:
    # сохраняем LL
    np.savetxt(f, LL, fmt='%d')
    # сохраняем LH
    f.write("LH:\n")
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    # сохраняем HL
    f.write("HL:\n")
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")
    # сохраняем HH
    f.write("HH:\n")
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")

haar_storage_size = os.path.getsize('haar_image.txt')

# 5. Сравните объем памяти исходного и сжатого изображения

In [74]:
print(f"Исходный размер: {pixel_storage_size} байт")
print(f"Размер после сжатия:: {haar_storage_size} байт")
print(f"Коэффициент сжатия: {pixel_storage_size / haar_storage_size:.2f}")

Исходный размер: 765222 байт
Размер после сжатия:: 191111 байт
Коэффициент сжатия: 4.00
